# Hackathon Dotz

##### Classificar as observações por (SUB-CATEGORIA | CATEGORIA | DEPARTAMENTO)

In [0]:
# Importação dos pacotes para Analise
 
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as sct
import statsmodels.api as sm
from google.colab import files  
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn import metrics
from sklearn.model_selection import train_test_split

In [26]:
#Carregar os dados para o google colab

uploaded = files.upload()

Saving Dados Gerais Datasets.txt to Dados Gerais Datasets.txt


In [0]:
# Importação dos dataset

df = pd.read_csv('Hackathon_Base_Treino_comdep.csv')
df2 = pd.read_csv('Hackathon_Base_Teste.csv')

### Análise dos dados

In [28]:
df['DESCRIÇÃO PARCEIRO'].str.slice(0,20)

0        PASTA INT VITAPOWER 
1        ESPONJA BETTANIN BRI
2        AGUA MIN SCHIN S/GAS
3        FITA DUPLA FACE C/SU
4        MASSA PIZZA ROMANHA 
                 ...         
22004    VINHO CHILENO ARESTI
22005    MOCHILA MASCULINA JU
22006    GEL FIX RADICAL PAST
22007    DOCE BANANA FALEX 16
22008    SHAMPOO TURMA XUXINH
Name: DESCRIÇÃO PARCEIRO, Length: 22009, dtype: object

### Etapa de Preparação do Dados

In [0]:
# Excluindo da descrição texto após os números, informações julgadas irrelevantes para a classificação.
df['DESC_AJUSTADA'] = df['DESCRIÇÃO PARCEIRO'].str.replace('[0-9].+', '', regex=True)
df['DESC_AJUSTADA_SUB'] = df['DESCRIÇÃO PARCEIRO'].str.replace('[0-9]+', '', regex=True)
# Dados de Teste
df2['DESC_AJUSTADA'] = df2['DESCRIÇÃO PARCEIRO'].str.replace('[0-9].+', '', regex=True)
df2['DESC_AJUSTADA_SUB'] = df2['DESCRIÇÃO PARCEIRO'].str.replace('[0-9]+', '', regex=True)

# Excluindo da descrição puntuação, informações julgadas irrelevantes para a classificação.
pontuacao=['.',',','-','+',':',';','&','+','/','!','#','%','(',')','  ']
for x in pontuacao:
  df['DESC_AJUSTADA'] = df['DESC_AJUSTADA'].str.replace(x, ' ')
  df['DESC_AJUSTADA_SUB'] = df['DESC_AJUSTADA_SUB'].str.replace(x, ' ')
  # Dados de Teste
  df2['DESC_AJUSTADA'] = df2['DESC_AJUSTADA'].str.replace(x, ' ')
  df2['DESC_AJUSTADA_SUB'] = df2['DESC_AJUSTADA_SUB'].str.replace(x, ' ')

In [30]:
df.head(2)

,DESCRIÇÃO PARCEIRO,SUB-CATEGORIA,CATEGORIA,DEPARTAMENTO,DESC_AJUSTADA,DESC_AJUSTADA_SUB
0,"PASTA INT VITAPOWER 1,005KG AMEND/SHOT",TRADICIONAL,CREME DE AMENDOIM,MERCEARIA DOCE,PASTA INT VITAPOWER,PASTA INT VITAPOWER KG AMEND SHOT
1,ESPONJA BETTANIN BRILHUS C/1,MULTIUSO,ESPONJA SINTÉTICA,CUIDADOS COM A COZINHA,ESPONJA BETTANIN BRILHUS C 1,ESPONJA BETTANIN BRILHUS C


In [0]:
# Variáveis importantes
# Palavras para retirar da análise
stop_words = ['em','sao','ao','de','da','do','com','para','c','kg','un','ml','pct','und','das',
              'no','ou','sem','gfa','jg','p/','la','1','2','3','4','5','6','7','8','9','0','a',
              'b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v',
              'x','w','y','z','kit','po','liq','cr','sh','pt','des','imp','lt','vd','az','esm',
              'cx','bc','lv','ad','gf','mix','rf','fr','bj','int','cd','cz','tp','ap','rs','caixa',
              'sao','bom','esp','extra','premium','pto','uso','sol','garrafa','brasil','pc','conj',
              'ph','gold','cj','mais','cor','facil','ice','classic','br','vm','tt','em','far','fix',
              'sc','dr','aco','nadir','romana','bauducco','original','pe','peso','sem','pro','sl',
              'und','seda','tradicional','arcor','teka','bas','dec','fina','palmolive','neutro',
              'sec','peq','baw','brinox','ext','fd','up','waw','seara','sadia','del','oetker',
              'royal','ret','boni','black','ln','new','preta','corttex','blue','big','ou','co','ex',
              'paf','pif','box','bdj','bestway','cinza','dourada','gluten','plasvale','protex','ype',
              'fs','for','no','das','lg','me','pvc','refil','tirolez','sm','friboi','ipa','pr','st',
              'xg','xxg','mc','nr','cp','ox','pa','pq','me','unid','ana','bi','di','gr']
stop_words_sub = ['em','sao','ao','de','da','do','com','para','c','kg','un','ml','pct','und','das',
              'no','ou','sem','gfa','jg','p/','la','1','2','3','4','5','6','7','8','9','0']
# Tipo acentuação
strip_accents = 'unicode'
# Tamanho da validação de teste
test_size = 0.3
# Interações no modelo
max_iter = 2000
# Tipo do vetor
tipo_vetor = TfidfVectorizer(stop_words=stop_words, strip_accents=strip_accents)
tipo_vetor_sub = TfidfVectorizer(stop_words=stop_words_sub, strip_accents=strip_accents)

### Etapa 1 - Classificando DEPARTAMENTO

In [0]:
#Criando a nova para classificar DEPARTAMENTO
base = df
 
# Selecionando apenas o item a ser classificado e o target do DF principal
base_data = base['DESC_AJUSTADA']
base_target = base['DEPARTAMENTO']
 
# Transformando a descrição em vetor
vetor = tipo_vetor
vetor_x = vetor.fit_transform(base_data)
 
# Dividindo dataset em treino e teste
x_train, x_test, y_train, y_test = train_test_split(vetor_x, base_target, test_size=test_size, random_state=27)
 
# Aqui nós configuramos o modelo para treinar o classificador
clf = PassiveAggressiveClassifier(max_iter=max_iter)

In [33]:
# Mostrar o tempo de processamento
%%time

# Executar o treinamento
clf.fit(x_train, y_train)

CPU times: user 818 ms, sys: 554 ms, total: 1.37 s
Wall time: 711 ms


PassiveAggressiveClassifier(C=1.0, average=False, class_weight=None,
                            early_stopping=False, fit_intercept=True,
                            loss='hinge', max_iter=2000, n_iter_no_change=5,
                            n_jobs=None, random_state=None, shuffle=True,
                            tol=0.001, validation_fraction=0.1, verbose=0,
                            warm_start=False)

In [34]:
# Avaliando a performance com predição
predicted = clf.predict(x_test)
 
print('\n###Indicadores Classificação###\n')
print(metrics.classification_report(y_test, predicted))


###Indicadores Classificação###

                          precision    recall  f1-score   support

              AUTOMOTIVO       0.94      0.84      0.89        89
                 AÇOUGUE       0.93      0.92      0.92       124
      BEBIDAS ALCOÓLICAS       0.92      0.99      0.95       405
  BEBIDAS NÃO ALCOÓLICAS       0.98      0.97      0.97       211
               BRINQUEDO       0.95      0.76      0.84        46
                 CALÇADO       0.99      0.98      0.99       189
      CAMA, MESA E BANHO       0.98      0.98      0.98       130
        CONGELADOS DOCES       0.96      0.95      0.95        74
     CONGELADOS SALGADOS       0.65      0.82      0.73        34
    CUIDADOS COM A BARBA       1.00      0.94      0.97        18
     CUIDADOS COM A CASA       0.90      0.92      0.91       134
  CUIDADOS COM A COZINHA       0.89      0.77      0.83        44
    CUIDADOS COM A ROUPA       0.94      0.99      0.96        96
 CUIDADOS COM MÃOS E PÉS       0.97      

In [35]:
# Predição em novos dados com o DF  de teste
base_novos = df2['DESC_AJUSTADA']

# Tranformando descrição em vetor
vetor_n = vetor.transform(base_novos)

#Previsão dos novos dados
predicted_novos = clf.predict(vetor_n)

# Adicionando a coluna para o novo DF
df_dep = df2[['DESCRIÇÃO PARCEIRO','DESC_AJUSTADA']]
df_dep['DEPARTAMENTO'] = predicted_novos
df_dep['DESC_AJUSTADA_SUB'] = df2['DESC_AJUSTADA_SUB']

# Novos Dados de teste incluindo o departamento
df_dep.head(2)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


,DESCRIÇÃO PARCEIRO,DESC_AJUSTADA,DEPARTAMENTO,DESC_AJUSTADA_SUB
0,SAL ROSA HIMALAIA C/ ALHO 500G,SAL ROSA HIMALAIA C ALHO,MERCEARIA SALGADA,SAL ROSA HIMALAIA C ALHO G
1,JG BOLA NATAL ACASA C/17 DR/PR/BC R952,JG BOLA NATAL ACASA C,DECORAÇÃO,JG BOLA NATAL ACASA C DR PR BC R


### Etapa 2 - Classificando CATEGORIA

In [0]:
# Criando a nova base para classificar CATEGORIA
base2 = df

# Selecionando apenas o item a ser classificado e o target do DF principal
base_data2 = base2['DEPARTAMENTO']+' '+base2['DESC_AJUSTADA']
base_target2 = base2['CATEGORIA']

# Transformando a descrição em vetor
vetor2 = tipo_vetor
vetor_x2 = vetor2.fit_transform(base_data2)

# Dividindo dataset em treino e teste
x_train2, x_test2, y_train2, y_test2 = train_test_split(vetor_x2, base_target2, test_size=test_size, random_state=27)

In [37]:
# Mostrar o tempo de processamento
%%time

# Aqui nós configuramos o modelo para treinar o classificador
clf2 = PassiveAggressiveClassifier(max_iter=max_iter)

# Executar o treinamento
clf2.fit(x_train2, y_train2)

CPU times: user 4.69 s, sys: 3.27 s, total: 7.96 s
Wall time: 4.05 s


In [38]:
# Avaliando a performance com predição
predicted2 = clf2.predict(x_test2)

print('\n###Indicadores Classificação Categoria###\n')
print(metrics.classification_report(y_test2, predicted2))


###Indicadores Classificação Categoria###

                                           precision    recall  f1-score   support

                               ABSORVENTE       1.00      1.00      1.00        20
                      ACESSÓRIOS DE BANHO       1.00      0.93      0.97        15
                 ACESSÓRIOS DE LAVANDERIA       1.00      1.00      1.00        16
                 ACESSÓRIOS DE MANUTENÇÃO       0.50      0.50      0.50         2
                        ACESSÓRIOS ELETRO       1.00      0.95      0.98        21
                      ACESSÓRIOS MÃO E PÉ       0.90      0.86      0.88        21
                   ACESSÓRIOS PARA CABELO       0.95      0.95      0.95        40
                           ACESSÓRIOS PET       0.90      0.82      0.86        22
                                 ADOÇANTE       1.00      1.00      1.00         5
                       AGUARDENTE/CACHAÇA       1.00      1.00      1.00        12
                                  ALGODÃO 

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [39]:
# Predição em novos dados utilizando o df_dep
base_novos2 = df_dep['DEPARTAMENTO']+' '+ df_dep['DESC_AJUSTADA']

# Tranformando descrição em vetor
vetor_n2 = vetor2.transform(base_novos2)

#Previsão dos novos dados
predicted_novos2 = clf2.predict(vetor_n2)

# Adicionando a coluna no novo DF
df_cat = df_dep
df_cat['CATEGORIA'] = predicted_novos2
df_cat['DESC_AJUSTADA_SUB'] = df_dep['DESC_AJUSTADA_SUB']

# Novos Dados de teste incluindo o departamento
df_cat.head(2)

,DESCRIÇÃO PARCEIRO,DESC_AJUSTADA,DEPARTAMENTO,DESC_AJUSTADA_SUB,CATEGORIA
0,SAL ROSA HIMALAIA C/ ALHO 500G,SAL ROSA HIMALAIA C ALHO,MERCEARIA SALGADA,SAL ROSA HIMALAIA C ALHO G,SAL
1,JG BOLA NATAL ACASA C/17 DR/PR/BC R952,JG BOLA NATAL ACASA C,DECORAÇÃO,JG BOLA NATAL ACASA C DR PR BC R,ENFEITES DE NATAL


### Etapa 3 - Classificando SUB-CATEGORIA

In [0]:
# Criando a nova para classificar CATEGORIA retirando apenas números
base3 = df

# Selecionando apenas o item a ser classificado e o target do DF principal
base_data3 = base3['DEPARTAMENTO']+' '+base3['CATEGORIA']+' '+base3['DESC_AJUSTADA_SUB']
base_target3 = base3['SUB-CATEGORIA']

# Transformando a descrição em vetor
vetor3 = tipo_vetor_sub
vetor_x3 = vetor3.fit_transform(base_data3)

# Dividindo dataset em treino e teste
x_train3, x_test3, y_train3, y_test3 = train_test_split(vetor_x3, base_target3, test_size=test_size, random_state=27)

In [41]:
# Mostrar o tempo de processamento
%%time

# Aqui nós treinamos o classificador
clf3 = PassiveAggressiveClassifier(max_iter=max_iter)

# Executar o treinamento
clf3.fit(x_train3, y_train3)

CPU times: user 28.5 s, sys: 19.9 s, total: 48.4 s
Wall time: 24.5 s


In [42]:
# Avaliando a performance com predição
predicted3 = clf3.predict(x_test3)

print('\n###Indicadores Classificação Categoria###\n')
print(metrics.classification_report(y_test3, predicted3))


###Indicadores Classificação Categoria###

                                          precision    recall  f1-score   support

                                  2 EM 1       0.00      0.00      0.00         1
                              20 VOLUMES       0.00      0.00      0.00         1
                              30 VOLUMES       0.00      0.00      0.00         0
                              40 VOLUMES       0.00      0.00      0.00         2
                                      4K       1.00      0.86      0.92         7
                                  A SECO       0.00      0.00      0.00         0
                                 ABACATE       1.00      0.50      0.67         2
                        ABACATE ORGÂNICO       0.00      0.00      0.00         0
                                 ABACAXI       0.73      0.89      0.80         9
                               ABOBRINHA       0.00      0.00      0.00         2
                      ABOBRINHA ORGÂNICA       0.50  

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [43]:
# Predição em novos dados
base_novos3 = df_cat['DEPARTAMENTO']+' '+df_cat['CATEGORIA']+' '+df_cat['DESC_AJUSTADA_SUB']

# Tranformando descrição em vetor
vetor_n3 = vetor3.transform(base_novos3)

#Previsão dos novos dados
predicted_novos3 = clf3.predict(vetor_n3)

# Adicionando a coluna
df_sub = df_cat
df_sub['SUB-CATEGORIA'] = predicted_novos3

# Novos Dados de teste incluindo o departamento
df_sub.head(10)

,DESCRIÇÃO PARCEIRO,DESC_AJUSTADA,DEPARTAMENTO,DESC_AJUSTADA_SUB,CATEGORIA,SUB-CATEGORIA
0,SAL ROSA HIMALAIA C/ ALHO 500G,SAL ROSA HIMALAIA C ALHO,MERCEARIA SALGADA,SAL ROSA HIMALAIA C ALHO G,SAL,SAL DO HIMALAIA
1,JG BOLA NATAL ACASA C/17 DR/PR/BC R952,JG BOLA NATAL ACASA C,DECORAÇÃO,JG BOLA NATAL ACASA C DR PR BC R,ENFEITES DE NATAL,BOLA DE NATAL
2,MIST PREP LIQ CHA VDE LINEA 56ML-CX LIMAO,MIST PREP LIQ CHA VDE LINEA,MERCEARIA DOCE,MIST PREP LIQ CHA VDE LINEA ML CX LIMAO,FARINÁCEO,MISTURAS
3,ALICATE CUT+ESPAT MUNDIAL SANDY AMARELO,ALICATE CUT ESPAT MUNDIAL SANDY AMARELO,CUIDADOS COM MÃOS E PÉS,ALICATE CUT ESPAT MUNDIAL SANDY AMARELO,ESMALTE,ESMALTE
4,MEIA PIZZA PEPPERONI UN,MEIA PIZZA PEPPERONI UN,PRATOS PRONTOS,MEIA PIZZA PEPPERONI UN,PIZZA,PEPPERONI
5,JARRA ACASA VIDRO CURVES OJG378,JARRA ACASA VIDRO CURVES OJG,UTILIDADES DE COZINHA,JARRA ACASA VIDRO CURVES OJG,UTENSÍLIOS DE VIDRO,JARRA/GARRAFA
6,FILME DE PVC CONSERV PACK 30M,FILME DE PVC CONSERV PACK,DESCARTÁVEIS,FILME DE PVC CONSERV PACK M,FILME PVC,FILME PVC
7,CESTO VIME CRAFTWORK S-25X15XH13,CESTO VIME CRAFTWORK S,LAVANDERIA,CESTO VIME CRAFTWORK S XXH,ORGANIZADOR,CESTO DE ROUPA
8,MAC BORTOLINI 500GR OVOS ESPAGUETE .,MAC BORTOLINI,MERCEARIA SALGADA,MAC BORTOLINI GR OVOS ESPAGUETE,MASSA SECA,ESPAGUETE
9,COND NOVEX OLEO OJON 300ML,COND NOVEX OLEO OJON,CUIDADOS COM O CABELO,COND NOVEX OLEO OJON ML,KIT CABELOS ADULTOS,KIT SHAMPOO E CONDICIONADOR


In [0]:
dotz = df_sub[['DESCRIÇÃO PARCEIRO', 'SUB-CATEGORIA', 'CATEGORIA']]
dotz.to_csv('dotz.csv')